In [18]:
%reload_ext autoreload
%autoreload
import pickle, os, json
import torch
from pathlib import Path, PureWindowsPath, PurePosixPath
from util.tools import load_config
import pandas as pd
from sklearn.metrics import classification_report

In [19]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = "cpu"

In [20]:
config_folder = "config/"
config = load_config(config_folder)
config.__dict__

{'hyperparams': <util.tools.config at 0x2a269058220>,
 'programsettings': <util.tools.config at 0x2a2690589a0>,
 'modelconfig': <util.tools.config at 0x2a268facb20>}

In [21]:
reports_folder_name = config.programsettings.REPORTS_DIR
data_folder = Path(reports_folder_name)
list_results_files = list(data_folder.glob('**/multi_model_*.pkl'))

all_results = []
all_data_source = []

for results_file in list_results_files:
#     print(results_file)
    with open(results_file,"rb") as f:
        lst_results = pickle.load(f)
#     print("File Name:", results_file, ' \n results: ', lst_results)
    
    for results in lst_results:
        max_seq = 0
        lr = 0
        train_batch = 0
        max_seq = ""
        model_name = ""
        class_weights = '[]'
        kernel_1 = 0
        kernel_2 = 0
        kernel_3 = 0
        if (len(results[0]) > 50):
#             print(results[0])
            results_json = json.loads(results[0])
            max_seq = results_json["hyperparams"]["MAX_SEQ_LENGTH"]
            lr = results_json["hyperparams"]["LEARNING_RATE"]
            train_batch = results_json["hyperparams"]["TRAIN_BATCH_SIZE"]
            model_name = results_json["programsettings"]["MODEL_NAME"]
            layers_freeze = results_json["hyperparams"]["NUM_BERT_LAYERS_FREEZE"]
            try:
                class_weights = results_json["hyperparams"]["LOSS_FN_CLASS_WEIGHTS"]
            
            except: 
                class_weights = '[]'
                
            try:
                kernel_1 = results_json["modelconfig"]["KERNEL_1"]
                kernel_2 = results_json["modelconfig"]["KERNEL_2"]
                kernel_3 = results_json["modelconfig"]["KERNEL_3"]
            
            except: 
                class_weights = '[]'

        result_store = [results_file, model_name,lr, train_batch,max_seq, layers_freeze, class_weights,kernel_1, kernel_2,kernel_3,results[1], \
                        results[2], results[3], results[4], results[5],results[6], results[15], results[16] ]
        all_results.append(result_store)
        
#       Creating list of all dev_labels and dev_preds, so that we can run classification reports for all of them together
        if (len(results) > 7):
            all_data_source.append([results[7],results[8], results[13], results[14], result_store])
                                
    
columns = ['file_name','model_name', 'lr', 'train_batch_size','train_max_seq','layers_freeze','class_weights','Kernel_1','Kernel_2','Kernel_3','train_loss', \
           'dev_loss', 'train_mcc', 'train_f1_score','dev_mcc','dev_f1_score', 'test_mcc', 'test_f1_score' ]

df_results = pd.DataFrame(all_results, columns = columns).sort_values(by='test_f1_score')
pd.set_option('display.max_colwidth', 100)
df_results

,file_name,model_name,lr,train_batch_size,train_max_seq,layers_freeze,class_weights,Kernel_1,Kernel_2,Kernel_3,train_loss,dev_loss,train_mcc,train_f1_score,dev_mcc,dev_f1_score,test_mcc,test_f1_score
14,reports\re\multi_model_experiment_results_2020-12-01 22_58_19_784900.pkl,BioBERT_CNN_fc,0.000013,12,256,8,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]",24,30,36,1.407102,1.409822,0.948392,0.744798,0.944330,0.741590,0.945090,0.742249
15,reports\re\multi_model_experiment_results_2020-12-02 00_15_00_467543.pkl,BioBERT_CNN_fc,0.000013,12,256,8,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]",24,30,36,1.406186,1.409095,0.949716,0.746246,0.945843,0.743710,0.947427,0.744703
23,reports\re\multi_model_experiment_results_2020-12-03 02_40_08_270805.pkl,BioBERT_fc,0.000013,12,384,8,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 4.0, 1.0]",36,48,60,1.390593,1.396370,0.974370,0.862349,0.968433,0.851543,0.967013,0.847240
16,reports\re\multi_model_experiment_results_2020-12-02 01_36_24_829782.pkl,BioBERT_CNN_fc,0.000013,12,256,8,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]",24,30,36,1.399931,1.402064,0.958933,0.856717,0.956166,0.853169,0.956139,0.853758
0,reports\re\multi_model_experiment_results_2020-12-01 21_42_10_815720.pkl,BioBERT_CNN_fc,0.000013,12,256,8,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]",24,30,36,1.399427,1.401749,0.959654,0.855617,0.956305,0.852596,0.957382,0.854498
17,reports\re\multi_model_experiment_results_2020-12-02 04_24_02_262718.pkl,BioBERT_CNN_fc,0.000013,12,256,8,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]",24,30,36,1.395771,1.399650,0.965372,0.862218,0.959540,0.855014,0.959412,0.855692
18,reports\re\multi_model_experiment_results_2020-12-02 06_25_18_129435.pkl,BioBERT_CNN_fc,0.000013,12,256,8,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]",4,30,36,1.396120,1.398611,0.964850,0.861088,0.961287,0.856947,0.959994,0.856914
11,reports\re\multi_model_experiment_results_2020-12-01 22_48_51_446054.pkl,BioBERT_fc,0.000010,12,128,10,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 1.0]",36,48,60,0.024447,0.130248,0.985398,0.984277,0.959827,0.947698,0.953923,0.942781
5,reports\re\multi_model_experiment_results_2020-12-01 22_48_51_446054.pkl,BioBERT_fc,0.000013,12,128,12,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]",36,48,60,0.023655,0.126698,0.985792,0.984727,0.958928,0.947031,0.955857,0.943452
13,reports\re\multi_model_experiment_results_2020-12-01 22_48_51_446054.pkl,BioBERT_fc,0.000010,12,128,10,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 8.0, 1.0]",36,48,60,0.024237,0.172904,0.985450,0.983735,0.960509,0.947396,0.957202,0.945593


### Let us generate Classification report for all the things we have available

In [22]:
for i in range(len(all_data_source)):
    dev_labels, dev_preds, test_preds, test_labels, model_config = all_data_source[i]
    print("\n \n \n",model_config)
#     print(classification_report(dev_labels,dev_preds ))
    print(classification_report(test_labels,test_preds ))
    


 
 
 [WindowsPath('reports/re/multi_model_experiment_results_2020-12-01 21_42_10_815720.pkl'), 'BioBERT_CNN_fc', 1.25e-05, 12, 256, 8, [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], 24, 30, 36, 1.3994271939717446, 1.4017490410132094, 0.9596541358440327, 0.8556167166995947, 0.9563050912588209, 0.8525956487620798, 0.9573819680978514, 0.8544979859717011]
              precision    recall  f1-score   support

           0       0.81      0.99      0.89      3464
           1       0.96      0.97      0.97      3723
           2       0.97      0.98      0.98      4274
           3       0.99      0.98      0.98      4090
           4       0.95      0.96      0.95       420
           5       0.98      0.93      0.95      3024
           6       0.98      0.97      0.97      2802
           7       0.00      0.00      0.00       775
           8       0.99      1.00      1.00     27438

    accuracy                           0.97     50010
   macro avg       0.85      0.86      0.85     5

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.96      0.97      0.97      3464
           1       0.98      0.98      0.98      3723
           2       0.99      0.98      0.99      4274
           3       0.99      0.99      0.99      4090
           4       0.95      0.96      0.96       420
           5       0.98      0.96      0.97      3024
           6       0.97      0.99      0.98      2802
           7       0.90      0.85      0.87       775
           8       1.00      1.00      1.00     27438

    accuracy                           0.99     50010
   macro avg       0.97      0.97      0.97     50010
weighted avg       0.99      0.99      0.99     50010


 
 
 [WindowsPath('reports/re/multi_model_experiment_results_2020-12-01 22_48_51_446054.pkl'), 'BioBERT_fc', 7.5e-06, 12, 128, 12, [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], 36, 48, 60, 0.025139369227550924, 0.11404036543174251, 0.9850065082053803, 0.983912820116114, 0.9599082883448715, 0.